## Predicting Churn with Neural Networks

In this notebook, I attempt to build a model based on <code>Keras</code> library and train it on the same fictional telecoms company data used in other parts of this project. The code below is mainly adapted from <code>TensorFlow</code>'s [tutorial](https://www.tensorflow.org/tutorials/structured_data/imbalanced_data) on how to handle imbalanced data.

I will first train a model without taking the different class weights into account. I will then compare the results to another model that is trained with the class imbalance accounted for.

In [ ]:
#loading the necessary python libraries
import pandas as pd
import numpy as np
import os
import tempfile

import seaborn as sns
sns.set(font_scale = 1.5)

import matplotlib.pyplot as plt
from matplotlib.colors import ListedColormap
%matplotlib inline
%config InlineBackend.figure_format = 'retina'

In [ ]:
#loading various libraries needed for preprocessing, modelling and evaluating
from sklearn.preprocessing import StandardScaler, OneHotEncoder, LabelEncoder
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import mean_squared_error, accuracy_score, recall_score, fbeta_score, \
                            precision_score, make_scorer, confusion_matrix, roc_curve, precision_recall_curve

import tensorflow as tf
from tensorflow import keras
from keras.wrappers.scikit_learn import KerasClassifier

In [ ]:
#loading the dataset into a pandas dataframe
customer_data = pd.read_csv('../data-sources/customer-churn/customer-churn.csv')
customer_data.head()

In [ ]:
#applying the same naming convention to all column names
customer_data.rename(columns={'customerID': 'CustomerID', 'gender': 'Gender', 'tenure': 'Tenure'}, inplace=True)

In [ ]:
#creating a LabelEncoder object
le = LabelEncoder()

#enoding the target variable into numerical binary
customer_data['Churn'] = le.fit_transform(customer_data['Churn'])

In [ ]:
#creating a function to convert numerical binary data into categorical binary data
def ColumnTransformer(cell):
    if cell == 0:
        return 'No'
    else:
        return 'Yes'

#applying the above function to the SeniorCitizen column
customer_data['SeniorCitizen'] = customer_data['SeniorCitizen'].apply(ColumnTransformer)

In [ ]:
#creating a function to shorten the vocabulary for easier visualisation
def ColumnTransformer(cell):
    if cell == 'Electronic check':
        return 'ElCh'
    elif cell == 'Mailed check':
        return 'MaCh'
    elif cell == 'Bank transfer (automatic)':
        return 'BaTr-A'
    else:
        return 'CrCa-A'
    
#applying the above function to the PaymentMethod column
customer_data['PaymentMethod'] = customer_data['PaymentMethod'].apply(ColumnTransformer)

In [ ]:
#removing all the customers who have been with the company for less than a month and deleting the CustomerID column
customer_data.drop(customer_data[customer_data['Tenure'] == 0].index, inplace=True)
customer_data.drop('CustomerID', axis=1, inplace=True)

#converting TotalCharges from object to float data type
customer_data['TotalCharges'] = customer_data['TotalCharges'].astype(float)

In [ ]:
#taking another look at the imbalance present in the data
print('Total: {}\n    Positive: {:.2f}\n    Negative: {:.2f}\n'.format(customer_data.shape[0],
                                                         customer_data['Churn'].value_counts(normalize=True)[1],
                                                         customer_data['Churn'].value_counts(normalize=True)[0]))

In [ ]:
##separating the target variable
target = customer_data.pop('Churn')

#dummifying the categorical features and removing any redundencies
customer_data_dum = pd.get_dummies(customer_data, columns=customer_data.select_dtypes(include='object').columns,
                                                   drop_first=True)

#adding the target variable back to the dummyfied data
customer_data_dum['Churn'] = target

In [ ]:
#splitting the data into different train, test and validation sets
train_data, test_data = train_test_split(customer_data_dum, test_size=0.2, stratify=customer_data_dum['Churn'])
train_data, validation_data = train_test_split(train_data, test_size=0.2, stratify=train_data['Churn'])

In [ ]:
#creating numpy arrays from the available labels and features
train_labels = np.array(train_data.pop('Churn'))
bool_train_labels = train_labels != 0
validation_labels = np.array(validation_data.pop('Churn'))
test_labels = np.array(test_data.pop('Churn'))

train_features = np.array(train_data)
validation_features = np.array(validation_data)
test_features = np.array(test_data)

In [ ]:
#creating a StandardScaler object and standardising train, validation and test sets
scaler = StandardScaler()

train_features = scaler.fit_transform(train_features)
validation_features = scaler.transform(validation_features)
test_features = scaler.transform(test_features)

print('Training labels shape:', train_labels.shape)
print('Validation labels shape:', validation_labels.shape)
print('Test labels shape:', test_labels.shape)

print('Training features shape:', train_features.shape)
print('Validation features shape:', validation_features.shape)
print('Test features shape:', test_features.shape)

In [ ]:
#separating the positive and negative churn instances into different dataframes
pos_data = pd.DataFrame(train_features[bool_train_labels], columns=train_data.columns)
neg_data = pd.DataFrame(train_features[~bool_train_labels], columns=train_data.columns)

#surveying two sample distributions 
sns.jointplot(pos_data['MonthlyCharges'], pos_data['Tenure'], kind='hex')
plt.suptitle('Positive distribution')

sns.jointplot(neg_data['MonthlyCharges'], neg_data['Tenure'], kind='hex')
plt.suptitle('Negative distribution')

#### Creating the Model and Defining the Metrics

In [ ]:
#outlining the different metrics to use for model evaluation
METRICS = [keras.metrics.TruePositives(name='tp'),
           keras.metrics.FalsePositives(name='fp'),
           keras.metrics.TrueNegatives(name='tn'),
           keras.metrics.FalseNegatives(name='fn'),
           keras.metrics.BinaryAccuracy(name='accuracy'),
           keras.metrics.Precision(name='precision'),
           keras.metrics.Recall(name='recall'),
           keras.metrics.AUC(name='auc'),
           keras.metrics.AUC(name='prc', curve='PR'),]

#creating a function that creates a simple neural network with two densly connected hidden layers, two dropout layer
#for regularization, and one output layer to return the probability of churn
def MakeModel(metrics=METRICS, output_bias=None):
    if output_bias is not None:
        output_bias = tf.keras.initializers.Constant(output_bias)
    model = keras.Sequential([keras.layers.Dense(24, activation='relu', input_shape=(train_features.shape[-1],)),
                              keras.layers.Dropout(0.5),
                              keras.layers.Dense(16, activation='relu'),
                              keras.layers.Dropout(0.4),
                              keras.layers.Dense(1, activation='sigmoid', bias_initializer = output_bias),
                             ])
    
    model.compile(optimizer=keras.optimizers.Adam(learning_rate=1e-3),
                  loss=keras.losses.BinaryCrossentropy(),
                  metrics=metrics)
    return model

In [ ]:
#defining the number of epochs and the batch size used during modelling
EPOCHS = 100
BATCH_SIZE = 150

#defining the early stopping criteria
early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_prc',
                                                  verbose=1,
                                                  patience=10,
                                                  mode='max',
                                                  restore_best_weights=True)

#building the model and printing out the summary
model = MakeModel()
model.summary()

In [ ]:
#plotting the model chart
tf.keras.utils.plot_model(model, show_shapes=True, rankdir='TB')

#### Correcting the Initial Bias

Given the imbalanced nature of the data, we know that the automatically generated initial bias might not be great. Here, I will first check the machine generated biases and the ncompare the Loss to a another model that reflects this imbalance in its output layer.

In [ ]:
#making predictions for the first 10 observations to test the model
model.predict(train_features[:10])

In [ ]:
#evaluating the Loss for these few predictions
results = model.evaluate(train_features, train_labels, batch_size=BATCH_SIZE, verbose=0)
print('Loss: {:0.4f}'.format(results[0]))

In [ ]:
#calculating the initial bias based on the present imbalance
neg, pos = np.bincount(customer_data_dum['Churn'])

total = neg + pos

initial_bias = np.log([pos/neg])
initial_bias

In [ ]:
#inserting the new bias figure into the model and making predictions
model = MakeModel(output_bias=initial_bias)
model.predict(train_features[:10])

In [ ]:
#evaluating the Loss for these latest predictions
results = model.evaluate(train_features, train_labels, batch_size=BATCH_SIZE, verbose=0)
print('Loss: {:0.4f}'.format(results[0]))

As we can see, using the correct initial bias has reduced the Loss. Next, I will quantify how much this bias initialisation actually helps.

In [ ]:
#saving the initial weights in a checkpoint file for future use
initial_weights = os.path.join(tempfile.mkdtemp(), 'initial_weights')
model.save_weights(initial_weights)

In [ ]:
#creating a model and setting the initial weights to zero 
model = MakeModel()
model.load_weights(initial_weights)
model.layers[-1].bias.assign([0.0])

#fitting the model and checking against the validation set
zero_bias_history = model.fit(train_features,
                              train_labels,
                              batch_size=BATCH_SIZE,
                              epochs=20,
                              validation_data=(validation_features, validation_labels),
                              verbose=0)

In [ ]:
#creating a model with the calculated initial weights
model = MakeModel()
model.load_weights(initial_weights)

#fitting the model and checking against the validation set
careful_bias_history = model.fit(train_features,
                                 train_labels,
                                 batch_size=BATCH_SIZE,
                                 epochs=20,
                                 validation_data=(validation_features, validation_labels),
                                 verbose=0)

In [ ]:
#creating a function to plot the loss metric over the range of epochs
def PlotLoss(history, label, color):
    plt.semilogy(history.epoch, history.history['loss'], color=color, label='Train ' + label)
    plt.semilogy(history.epoch, history.history['val_loss'], color=color, label='Validation ' + label, ls='--')
    plt.xlabel('Epoch')
    plt.ylabel('Loss')

In [ ]:
#plotting the loss and validation loss for each weight bias
plt.figure(figsize=(10, 6))
PlotLoss(zero_bias_history, 'Zero Bias', 'black')
PlotLoss(careful_bias_history, 'Careful Bias', 'crimson')

As we can see, the difference in this case isn't significant. However, using the correct initial weight is an important step in dealing with imbalanced data. Therefore, I will be using these initial weights from now on, regardless of their impact in this particular instance.

#### Class Weights

In this section, I will first train and evaluate a model without passing class weights, followed by training and evaluating a model that takes class weights into account.

In [ ]:
#training and validating a model with initial weights and predefined number of epochs 
model = MakeModel()
model.load_weights(initial_weights)
baseline_history = model.fit(train_features,
                             train_labels,
                             batch_size=BATCH_SIZE,
                             epochs=EPOCHS,
                             callbacks=[early_stopping],
                             validation_data=(validation_features, validation_labels))

In [ ]:
#creating a function to plot different model metrics against the number of epochs
def PlotMetrics(history):
    metrics = ['loss', 'prc', 'precision', 'recall']
    plt.figure(figsize=(15, 15))
    for n, metric in enumerate(metrics):
        name = metric.replace('_', ' ').capitalize()
        plt.subplot(2, 2, n+1)
        plt.plot(history.epoch, history.history[metric], color='black', label='Train')
        plt.plot(history.epoch, history.history['val_' + metric], color='black', ls='--', label='Validation')
        plt.xlabel('Epoch')
        plt.ylabel(name)
        if metric == 'loss':
            plt.ylim([0, plt.ylim()[1]])
        elif metric == 'auc':
            plt.ylim([0.8, 1])
        else:
            plt.ylim([0, 1])
        plt.legend()

In [ ]:
#plotting the Loss, PRC, Precision, and Recall
PlotMetrics(baseline_history)

In [ ]:
#making predictions on train and test sets
train_predictions_baseline = model.predict(train_features, batch_size=BATCH_SIZE)
test_predictions_baseline = model.predict(test_features, batch_size=BATCH_SIZE)

In [ ]:
#creating a function to plot the confusion matricx
def ConfMatrix(labels, predictions, p=0.5):
    cm = confusion_matrix(labels, predictions > p)
    plt.figure(figsize=(8, 7))
    sns.heatmap(cm, annot=True, fmt='d')
    plt.title('Confusion matrix @{:.2f}'.format(p))
    plt.ylabel('Actual label')
    plt.xlabel('Predicted label')

In [ ]:
#calculating the model metrics and plotting the confusion matrix
baseline_results = model.evaluate(test_features, test_labels, batch_size=BATCH_SIZE, verbose=0)

for name, value in zip(model.metrics_names, baseline_results):
    print(name, ': ', value)

ConfMatrix(test_labels, test_predictions_baseline)

In [ ]:
#creating a function to plot the ROC
def PlotROC(name, labels, predictions, **kwargs):
    fp, tp, _ = roc_curve(labels, predictions)
    plt.plot(100*fp, 100*tp, label=name, lw=2, **kwargs)
    plt.xlabel('False positives [%]')
    plt.ylabel('True positives [%]')
    plt.xlim([-0.5, 100.5])
    plt.ylim([-0.5, 100.5])
    plt.grid(True)
    ax = plt.gca()
    ax.set_aspect('equal')

In [ ]:
#plotting the ROC for train and test predictions
plt.figure(figsize=(8, 8))
PlotROC('Train Baseline', train_labels, train_predictions_baseline, color='black')
PlotROC('Test Baseline', test_labels, test_predictions_baseline, color='black', ls='--')
plt.legend(loc='lower right')

In [ ]:
#creating a function to plot the PRC
def PlotPRC(name, labels, predictions, **kwargs):
    precision, recall, _ = precision_recall_curve(labels, predictions)
    plt.plot(precision, recall, label=name, lw=2, **kwargs)
    plt.xlabel('Recall')
    plt.ylabel('Precision')
    plt.xlim([-0.05, 1.05])
    plt.ylim([-0.05, 1.05])
    plt.grid(True)
    ax = plt.gca()
    ax.set_aspect('equal')

In [ ]:
#plotting the PRC for train and test predictions
plt.figure(figsize=(8, 8))
PlotPRC('Train Baseline', train_labels, train_predictions_baseline, color='black')
PlotPRC('Test Baseline', test_labels, test_predictions_baseline, color='black', ls='--')
plt.legend(loc='lower left')

Let's calcualte the weight for each class by multiplying the inverse of class population by half of total population.

In [ ]:
#calculating class weights
weight_for_0 = (1/neg)*(total/2)
weight_for_1 = (1/pos)*(total/2)

class_weight = {0: weight_for_0, 1: weight_for_1}

print('Weight for class 0: {:.2f}'.format(weight_for_0))
print('Weight for class 1: {:.2f}'.format(weight_for_1))

In [ ]:
#creating and training a model using calculated initial weights and also class weights
weighted_model = MakeModel()
weighted_model.load_weights(initial_weights)
weighted_history = weighted_model.fit(train_features,
                                      train_labels,
                                      batch_size=BATCH_SIZE,
                                      epochs=EPOCHS,
                                      callbacks=[early_stopping],
                                      validation_data=(validation_features, validation_labels),
                                      class_weight=class_weight)

In [ ]:
#plotting the evaluation metrics
PlotMetrics(weighted_history)

In [ ]:
#making predictions on train and test sets
train_predictions_weighted = weighted_model.predict(train_features, batch_size=BATCH_SIZE)
test_predictions_weighted = weighted_model.predict(test_features, batch_size=BATCH_SIZE)

In [ ]:
#calculating the model metrics and plotting the confusion matrix
weighted_results = weighted_model.evaluate(test_features, test_labels, batch_size=BATCH_SIZE, verbose=0)

for name, value in zip(weighted_model.metrics_names, weighted_results):
    print(name, ': ', value)

ConfMatrix(test_labels, test_predictions_weighted)

In [ ]:
#plotting the ROC for both models to compare the effect of explicitly supplying the class weigths
plt.figure(figsize=(8, 8))

PlotROC('Train Baseline', train_labels, train_predictions_baseline, color='black')
PlotROC('Test Baseline', test_labels, test_predictions_baseline, color='black', ls='--')

PlotROC('Train Weighted', train_labels, train_predictions_weighted, color='crimson')
PlotROC('Test weighted', test_labels, test_predictions_weighted, color='crimson', ls='--')

plt.legend(loc='lower right')

In [ ]:
#plotting the PRC for both models to compare the effect of explicitly supplying the class weigths
plt.figure(figsize=(8, 8))

PlotPRC('Train Baseline', train_labels, train_predictions_baseline, color='black')
PlotPRC('Test Baseline', test_labels, test_predictions_baseline, color='black', ls='--')

PlotPRC('Train Weighted', train_labels, train_predictions_weighted, color='crimson')
PlotPRC('Test Weighted', test_labels, test_predictions_weighted, color='crimson', ls='--')

plt.legend(loc='lower left')

Looking at these plots, it seems that supplying class weights doesn't have a noticable effect on either the ROC or PRC. Perhaps the present imbalance is not large enough to be significant.

Let's now look at one final metric, F1-beta score, as a way of comparing this neural network model with the more traditional models I trained in another [notebook](churn-predictor.ipynb).

In [ ]:
#converting the probabilities to class predictions and calculating the F1-beta score
predicted_labels = test_predictions_weighted > 0.5
fbeta_score(test_labels, predicted_labels, beta=0.85, pos_label=1)

Even though the difference isn't large, this score compares favourably with the one achieved by the <code>VotingClassifier</code>. It is also important to note that this score can potentially be improved upon by careful optimisation, something that hasn't been considered in this project and can be the basis of future work.

In [ ]:
def MakeTuner(metrics=METRICS, output_bias=initial_bias, lr=1e-3, dr=0.5, top_neurons=5, bottom_neurons=5):
    if output_bias is not None:
        output_bias = tf.keras.initializers.Constant(output_bias)
    model = keras.Sequential([keras.layers.Dense(top_neurons, activation='relu', input_shape=(train_features.shape[-1],)),
                              keras.layers.Dropout(dr),
                              keras.layers.Dense(bottom_neurons, activation='relu'),
                              keras.layers.Dropout(dr),
                              keras.layers.Dense(1, activation='sigmoid', bias_initializer = output_bias),
                             ])
    
    model.compile(optimizer=keras.optimizers.Adam(learning_rate=lr),
                  loss=keras.losses.BinaryCrossentropy(),
                  metrics=metrics)
    return model

In [ ]:
f1_beta = make_scorer(fbeta_score, beta=0.85)

In [ ]:
model_tuner = KerasClassifier(build_fn=MakeTuner, verbose=0)

param_grid = {'batch_size': [20, 100, 500],
              'epochs': [50, 100, 150],
              'lr': [1e-4, 1e-3],
              'dr': [0.4, 0.5],
              'top_neurons': [10, 25],
              'bottom_neurons': [10, 25],
              'class_weight': [class_weight]}

model_tuner_gs = GridSearchCV(model_tuner, param_grid, cv=3, scoring=f1_beta)
tuning_result = model_tuner_gs.fit(train_features, train_labels)

print("Cross-validated F1-beta score: %f using %s" % (tuning_result.best_score_, tuning_result.best_params_))

In [ ]:
train_predictions_tuned = model_tuner_gs.predict(train_features)
test_predictions_tuned = model_tuner_gs.predict(test_features)

In [ ]:
ConfMatrix(test_labels, test_predictions_tuned)